In [1]:
from utils.df_handle import *

Default bq project:  spatial-vision-343005.biteam


In [2]:
sql = \
"""
SELECT
CONCAT(a.BranchID,a.BatNbr,a.OrderNbr,Sequence) as pk,
b.LUpd_DateTime as Delivery_Date
from OM_Delivery a
LEFT JOIN
(
select * from 
(
	select
	DISTINCT
	Status,
	BranchID,
	BatNbr,
	OrderNbr,
	LUpd_DateTime,
	ROW_NUMBER() OVER (PARTITION BY Status,BranchID,BatNbr,OrderNbr  ORDER BY LUpd_DateTime DESC ) AS RowNum
	FROM OM_DeliHistory
	where Status = 'C'
) #a
where RowNum = 1
) b

ON a.BranchID = b.BranchID
AND a.BatNbr = b.BatNbr
AND a.OrderNbr = b.OrderNbr
"""

In [3]:
df = get_ms_df(sql)

In [4]:
bq_values_insert(df, "approvedate", 3)

In [3]:
FINAL = pd.read_pickle("file.pk")

In [4]:
pk = ['macongtycn', 'ngaychungtu', 'sodondathang', 'masanpham', 'solo', 'lineref', 'soluong']

In [10]:
FINAL[checkdup(FINAL, 2, subset=pk)].to_clipboard()

In [2]:
sql = \
f"""
DECLARE CHANGED_DATE DATE DEFAULT '2023-01-02';

with cur_version as
(
  select
  custid,
  version_id,
  version_id -1 as version_lv,
  lupd_datetime,
  lupd_user,
  case 
  when CHR(32) = new_value then "Unknow" 
  when new_value is null then "Unknow"  else new_value end as new_value,
  changetype
  from biteam.d_master_custhis
  UNPIVOT (new_value for changetype in (branchid, salessystem, channel, shoptype, hcoid, hcotype, classid, checkterm, custname, addr1, attn, country, district, emailaddr, fax, phone, slsperid, state, status, terms, territory, zip, establishdate, refcustid, inactive, ward, businessname, market, billmarket, oricustid, generalcustid, paymentsform, genorders, batchexpform, custidpublic, shoperid, isagency, agencyid, taxdeclaration, stocksales, businessscope, legalname, legaldate, chargereceive, chargepayment, chargephar))
  where date(lupd_datetime) = CHANGED_DATE and version_id > 1
  -- and custid = '001733'

)
,
las_version as
(
select
custid,
version_id,
-- version_id -1 as version_lv,
-- lupd_datetime,
  case 
  when CHR(32) = old_value then "Unknow" 
  when old_value is null then "Unknow"  else old_value end as old_value,
changetype
from biteam.d_master_custhis
UNPIVOT (old_value for changetype in (branchid, salessystem, channel, shoptype, hcoid, hcotype, classid, checkterm, custname, addr1, attn, country, district, emailaddr, fax, phone, slsperid, state, status, terms, territory, zip, establishdate, refcustid, inactive, ward, businessname, market, billmarket, oricustid, generalcustid, paymentsform, genorders, batchexpform, custidpublic, shoperid, isagency, agencyid, taxdeclaration, stocksales, businessscope, legalname, legaldate, chargereceive, chargepayment, chargephar))
-- where custid = '001733'
)

select 
cur.*,
old_value,
from 
cur_version cur
left join las_version las
ON cur.custid = las.custid
AND cur.version_lv = las.version_id
AND cur.changetype = las.changetype
where new_value != old_value
"""

In [3]:
df = get_bq_df(sql)

In [4]:
df.head()

,custid,version_id,version_lv,lupd_datetime,lupd_user,new_value,changetype,old_value
0,TD30O1538,3,2,2023-01-02 18:09:34.970000+00:00,admin,PCL,channel,OTC
1,TD30O1538,3,2,2023-01-02 18:09:34.970000+00:00,admin,PCL,shoptype,PK
2,TD30O1538,3,2,2023-01-02 18:09:34.970000+00:00,admin,PKCK,hcoid,PK
3,TD30O1538,3,2,2023-01-02 18:09:34.970000+00:00,admin,NTXQPK,hcotype,PKK
4,TD30O1538,3,2,2023-01-02 18:09:34.970000+00:00,admin,"01,02,04,05,06",businessscope,UnKnow


In [6]:
df = pd.read_pickle("2023-01-06_Torder.pk")

In [11]:
del(df)

In [9]:
TOrder = df[df.OrderNbr == 'DL0-0123-00421']

In [10]:
# df[df.OrderNbr == 'DL0-0123-00421'].to_clipboard()

In [13]:
path = ''

In [49]:
lst = ['BranchID','OrderNbr','OMOrder','PaymentsForm','TermsID','CustID','InvoiceCustID','Version']

TCustomer = TOrder[lst].copy()

# vc(TCustomer, 'PaymentsForm')

TCustomer.PaymentsForm = np.where(TCustomer.PaymentsForm.isin(['B','C']),'TM','CK')

# vc(TCustomer, 'PaymentsForm')

c_sql = \
"""
SELECT 
CustId as CustID,
CustName,
Channel,
ShopType,
Territory,
Addr1,
Ward,
District,
State,
Attn,
Phone
from AR_Customer
"""
c = get_ms_df(c_sql)

cl_sql = \
"""
SELECT 
CustID,
CustName,
Channel,
ShopType,
Territory,
Addr1,
Ward,
District,
State,
Attn,
Phone
from AR_Customer
"""
cl = get_ms_df(cl_sql)

# TCustomer.shape

# cl.columns

TCustomer1 = TCustomer.merge(cl, how = 'left', on=['Version'])

# TCustomer1.shape

lst = TCustomer1.columns

TCustomer2 = TCustomer1.merge(c, how = 'inner', on=['CustID'], suffixes=['','_c'])

TCustomer2.CustName.fillna(TCustomer2.CustName_c, inplace=True)

TCustomer2.Channel.fillna(TCustomer2.Channel_c, inplace=True)

TCustomer2.ShopType.fillna(TCustomer2.ShopType_c, inplace=True)

TCustomer2.Territory.fillna(TCustomer2.Territory_c, inplace=True)

TCustomer2.Addr1.fillna(TCustomer2.Addr1_c, inplace=True)

TCustomer2.Ward.fillna(TCustomer2.Ward_c, inplace=True)

TCustomer2.District.fillna(TCustomer2.District_c, inplace=True)

TCustomer2.State.fillna(TCustomer2.State_c, inplace=True)

TCustomer2.Attn.fillna(TCustomer2.Attn_c, inplace=True)

TCustomer2.Phone.fillna(TCustomer2.Phone_c, inplace=True)

TCustomer2 = TCustomer2[lst]

aci_sql = \
"""
SELECT
CustIDInvoice as InvoiceCustID,
Phone as PhoneCustInvc
from AR_CustomerInvoice
"""
aci = get_ms_df(aci_sql)

TCustomer3 = TCustomer2.merge(aci, how = 'left', on=['InvoiceCustID'])

# TCustomer3.columns

TCustomer = TCustomer3

del(TCustomer1)
del(TCustomer2)
del(TCustomer3)

# TCustomer.shape

TCustomer.drop_duplicates(inplace=True)

# TCustomer.shape

# Output

o = TOrder
# del(TOrder)


In [50]:
o.head()

,BranchID,SlsperID,OrderNbr,OMOrder,DeliveryUnit,CustID,InvoiceCustID,InvcNbr,InvcNote,Version,...,DueDate,OpeiningOrderAmt,OrdAmtRelease,DeliveredOrderAmt,ReturnOrdAmt,ReceiveAmt,Reason,DebConfirmAmt,DebConfirmAmtRelease,PaymentsForm
359477,HCM001,MR0977,DL0-0123-00421,HL0-0123-00355,PN,004208,007938,00001280,1C23TMR,111378,...,2023-01-07,0.0,500000.0,0,0,0,,0,0,B


In [51]:
tc = TCustomer
# del(TCustomer)

sys_sql = """select CpnyID as BranchID, CpnyName from SYS_Company"""
sys = get_ms_df(sys_sql)

o.columns

o = o.merge(sys, how = "inner", on="BranchID")

In [52]:
o.head()

,BranchID,SlsperID,OrderNbr,OMOrder,DeliveryUnit,CustID,InvoiceCustID,InvcNbr,InvcNote,Version,...,OpeiningOrderAmt,OrdAmtRelease,DeliveredOrderAmt,ReturnOrdAmt,ReceiveAmt,Reason,DebConfirmAmt,DebConfirmAmtRelease,PaymentsForm,CpnyName
0,HCM001,MR0977,DL0-0123-00421,HL0-0123-00355,PN,004208,007938,00001280,1C23TMR,111378,...,0.0,500000.0,0,0,0,,0,0,B,CÔNG TY CỔ PHẦN TẬP ĐOÀN MERAP - CN HỒ CHÍ MINH


In [53]:
u_sql = """
select Username as SlsperID, FirstName,
Position =
CASE
WHEN u.UserTypes LIKE '%LOG%' THEN'LOG'
WHEN u.Position IN ( 'D', 'SD', 'AD', 'RD' ) AND u.UserTypes NOT LIKE '%LOG%' THEN 'MDS'
WHEN u.UserTypes LIKE '%CS%' THEN 'CS' 
WHEN u.Position IN ( 'S', 'SS', 'AM', 'RM' ) THEN 'P.BH' 
ELSE u.Position END

from Users as u"""
u = get_ms_df(u_sql)

o = o.merge(u, how = "inner", on="SlsperID")

In [54]:
o.head()

,BranchID,SlsperID,OrderNbr,OMOrder,DeliveryUnit,CustID,InvoiceCustID,InvcNbr,InvcNote,Version,...,DeliveredOrderAmt,ReturnOrdAmt,ReceiveAmt,Reason,DebConfirmAmt,DebConfirmAmtRelease,PaymentsForm,CpnyName,FirstName,Position
0,HCM001,MR0977,DL0-0123-00421,HL0-0123-00355,PN,004208,007938,00001280,1C23TMR,111378,...,0,0,0,,0,0,B,CÔNG TY CỔ PHẦN TẬP ĐOÀN MERAP - CN HỒ CHÍ MINH,Lê Minh Tú,MDS


In [55]:
o = o.merge(tc, how = "inner", on=["BranchID","OMOrder","OrderNbr"], suffixes=['',"_tc"])
o.head()

,BranchID,SlsperID,OrderNbr,OMOrder,DeliveryUnit,CustID,InvoiceCustID,InvcNbr,InvcNote,Version,...,Channel,ShopType,Territory,Addr1,Ward,District,State,Attn,Phone,PhoneCustInvc
0,HCM001,MR0977,DL0-0123-00421,HL0-0123-00355,PN,004208,007938,00001280,1C23TMR,111378,...,TP,PMC,8,79 Lê Đình Thám,4708,,30,Anh Cường,093 7321652,


In [56]:
o = o.merge(tc, how = "inner", on=["BranchID","OMOrder","OrderNbr"], suffixes=['',"_tc"])

In [57]:
o.head()

,BranchID,SlsperID,OrderNbr,OMOrder,DeliveryUnit,CustID,InvoiceCustID,InvcNbr,InvcNote,Version,...,Channel_tc,ShopType_tc,Territory_tc,Addr1_tc,Ward_tc,District_tc,State_tc,Attn_tc,Phone_tc,PhoneCustInvc_tc
0,HCM001,MR0977,DL0-0123-00421,HL0-0123-00355,PN,004208,007938,00001280,1C23TMR,111378,...,TP,PMC,8,79 Lê Đình Thám,4708,,30,Anh Cường,093 7321652,


In [58]:
ste_sql = """select Descr as Territory_Name, Territory from SI_Territory"""
ste = get_ms_df(ste_sql)

o = o.merge(ste, how = "inner", on=["Territory"], suffixes=['',"_ste"])

In [59]:
o.head()

,BranchID,SlsperID,OrderNbr,OMOrder,DeliveryUnit,CustID,InvoiceCustID,InvcNbr,InvcNote,Version,...,ShopType_tc,Territory_tc,Addr1_tc,Ward_tc,District_tc,State_tc,Attn_tc,Phone_tc,PhoneCustInvc_tc,Territory_Name
0,HCM001,MR0977,DL0-0123-00421,HL0-0123-00355,PN,004208,007938,00001280,1C23TMR,111378,...,PMC,8,79 Lê Đình Thám,4708,,30,Anh Cường,093 7321652,,Hồ Chí Minh 1


In [60]:
sta_sql = """select Descr as State_Name, State from SI_State"""
sta = get_ms_df(sta_sql)

o = o.merge(sta, how = "inner", on=["State"], suffixes=['',"_sta"])

In [61]:
o.head()

,BranchID,SlsperID,OrderNbr,OMOrder,DeliveryUnit,CustID,InvoiceCustID,InvcNbr,InvcNote,Version,...,Territory_tc,Addr1_tc,Ward_tc,District_tc,State_tc,Attn_tc,Phone_tc,PhoneCustInvc_tc,Territory_Name,State_Name
0,HCM001,MR0977,DL0-0123-00421,HL0-0123-00355,PN,004208,007938,00001280,1C23TMR,111378,...,8,79 Lê Đình Thám,4708,,30,Anh Cường,093 7321652,,Hồ Chí Minh 1,Thành phố Hồ Chí Minh


In [62]:
o.to_clipboard()

In [47]:
# ste_sql = """select Descr as Territory_Name, Territory from SI_Territory"""
# ste = get_ms_df(ste_sql)

# o = o.merge(ste, how = "inner", on=["Territory"], suffixes=['',"_ste"])



sd_sql = """select Name as District_Name, District, State from SI_District"""
sd = get_ms_df(sd_sql)

o = o.merge(sd, how = "inner", on=["District","State"], suffixes=['',"_sd"])

In [48]:
o.head()

,BranchID,SlsperID,OrderNbr,OMOrder,DeliveryUnit,CustID,InvoiceCustID,InvcNbr,InvcNote,Version,...,Addr1_tc,Ward_tc,District_tc,State_tc,Attn_tc,Phone_tc,PhoneCustInvc_tc,Territory_Name,State_Name,District_Name


In [23]:
sw_sql = """select Name as Ward_Name, Ward, State, District from SI_Ward"""
sw = get_ms_df(sw_sql)

o = o.merge(sw, how = "left", on=["Ward", "State","District"], suffixes=['',"_sw"])

atr_sql = """select Code as DeliveryUnit, Descr as DeliveryUnit_Name from AR_Transporter"""
atr = get_ms_df(atr_sql)

o = o.merge(atr, how = "left", on=["DeliveryUnit"], suffixes=['',"_atr"])

ts_sql = """SELECT SlsperID, BranchID, SupID, ASM, RSMID FROM dbo.fr_ListSaleByData('MR2523');"""
ts = get_ms_df(ts_sql)

o = o.merge(ts, how = "left", on=["SlsperID","BranchID"], suffixes=['',"_ts"])

uu_sql = """SELECT FirstName as SupName, UserName as SupID FROM Users"""
uu = get_ms_df(uu_sql)

o = o.merge(uu, how = "left", on=["SupID"], suffixes=['',"_uu"])

asm_sql = """SELECT FirstName as ASMName, UserName as ASM FROM Users"""
asm = get_ms_df(asm_sql)

o = o.merge(asm, how = "left", on=["ASM"], suffixes=['',"_asm"])

rsm_sql = """SELECT FirstName as RSMName, UserName as RSMID FROM Users"""
rsm = get_ms_df(rsm_sql)

o = o.merge(rsm, how = "left", on=["RSMID"], suffixes=['',"_rsm"])

st_sql = """SELECT DueType, DueIntrv, TermsID as TermsID_tc FROM SI_Terms"""
st = get_ms_df(st_sql)

o = o.merge(st, how = "left", on=["TermsID_tc"], suffixes=['',"_st"])

stt_sql = """SELECT Descr as Terms, TermsID FROM SI_Terms"""
stt = get_ms_df(stt_sql)

o = o.merge(stt, how = "left", on=["TermsID"], suffixes=['',"_stt"])

g = pd.read_csv(path+"gdo.csv")

o = o.merge(g, how = "left", on=["PaymentsForm_tc","TermsID"], suffixes=['',"_g"])

o.DebtInCharge.fillna("CS", inplace=True)

In [24]:
# o.head()

,OrderNbr,OMOrder,CustID,InvoiceCustID,InvcNbr,InvcNote,Version,OrderDate,DateOfOrder,DeliveryTime,...,ASM,RSMName,RSMID,DueType,DueIntrv,TermsID_tc,Terms,TermsID,PaymentsForm_tc,DebtInCharge


In [18]:


# df = pd.DataFrame(o.columns)

# df.to_clipboard()

lst = [
"BranchID",
"SlsperID",
"OrderNbr",
"CustID",
"OrderDate",
"DateOfOrder",
"DeliveryTime",
"TermsID",
"DueDate",
"CpnyName",
"FirstName",
"Position",
"PaymentsForm_tc",
"InvoiceCustID_tc",
"CustName",
"Channel",
"ShopType",
"Addr1",
"Attn",
"Phone",
"Territory_Name",
"State_Name",
"District_Name",
"Ward_Name",
"DeliveryUnit_Name",
"SupName",
"ASMName",
"RSMName",
"DueType",
"DueIntrv",
"Terms",
"DebtInCharge",
"OpeiningOrderAmt",
"OrdAmtRelease",
"DeliveredOrderAmt",
"ReturnOrdAmt",
"ReceiveAmt",
"Reason",
"DebConfirmAmt",
"DebConfirmAmtRelease",
]

o = o[lst]

lst = [
"BranchID",
"SlsperID",
"OrderNbr",
"CustID",
"OrderDate",
"DateOfOrder",
"DeliveryTime",
"TermsID",
"DueDate",
"CpnyName",
"FirstName",
"Position",
"PaymentsForm",
"InvoiceCustID",
"CustName",
"Channel",
"ShopType",
"StreetName",
"Attn",
"Phone",
"Territory",
"State",
"District",
"Ward",
"DeliveryUnit",
"SupName",
"ASMName",
"RSMName",
"DueType",
"DueIntrv",
"Terms",
"DebtInCharge",
"OpeiningOrderAmt",
"OrdAmtRelease",
"DeliveredOrderAmt",
"ReturnOrdAmt",
"ReceiveAmt",
"Reason",
"DebConfirmAmt",
"DebConfirmAmtRelease",
]

o.columns = lst

# o.dtypes

dk1 = o.DueType == "D"
dk2 = o.DueIntrv.isin([1.0, 2.0])

# vc(o, "DueIntrv")

o['TermsType'] = np.where(dk1&dk2, "Thanh Toán Ngay", "Cho nợ")

drop_cols(o, ['DueType','DueIntrv'])

o.Terms.fillna(o.TermsID,inplace=True)

drop_cols(o, ['TermsID'])

# o.columns

# vc(o,"PaymentsForm")

wsc = pd.read_csv(path+"wsc.csv")

# wsc.head()

o = o.merge(wsc, how = "left", on=["PaymentsForm","ShopType"], suffixes=['',"_wsc"])

In [19]:
o.head()

,BranchID,SlsperID,OrderNbr,CustID,OrderDate,DateOfOrder,DeliveryTime,DueDate,CpnyName,FirstName,...,DeliveredOrderAmt,ReturnOrdAmt,ReceiveAmt,Reason,DebConfirmAmt,DebConfirmAmtRelease,TermsType,ShopType,PaymentsForm,No_Count


In [17]:

dk = o.No_Count.isna()

o = o[dk].copy()

drop_cols(o, "No_Count")

group_lst = \
[
"BranchID",
"SlsperID",
"OrderNbr",
"CustID",
"OrderDate",
"DateOfOrder",
"DueDate",
"CpnyName",
"FirstName",
"Position",
"PaymentsForm",
"InvoiceCustID",
"CustName",
"Channel",
"ShopType",
"StreetName",
"Attn",
"Phone",
"Territory",
"State",
"District",
"Ward",
"DeliveryUnit",
"SupName",
"ASMName",
"RSMName",
"Terms",
"DebtInCharge",
"TermsType",
]

group_dict = \
{
# "DeliveryTime":np.max,
# "Reason":np.max,
"OpeiningOrderAmt":np.sum,
"OrdAmtRelease":np.sum,
"DeliveredOrderAmt":np.sum,
"ReturnOrdAmt":np.sum,
"ReceiveAmt":np.sum,
"DebConfirmAmt":np.sum,
"DebConfirmAmtRelease":np.sum
}

o = pivot(o, group_lst, group_dict)

In [4]:
df.to_excel("data.xlsx", index=False)

In [2]:
df = pd.read_pickle("df.pickle")

In [4]:
df.to_excel("snapshot.xlsx", index=False)

In [15]:
df = get_ps_df("select masanpham, nhomsp from d_nhom_sp where nhomsp = 'PHA NAM'")

In [16]:
df.to_pickle("d_nhom_sp.df")

In [4]:
# dskh = pd.read_excel("dskh_t11.xlsx")

In [5]:
# bq_values_insert(dskh, "d_phanloai_chamsoc_kh", 3)

In [1]:
data = {}

In [2]:
len(data)

0

In [5]:
#lấy data danh sách report
df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vSuXl7AnpSA5j_KrgFO2zhLn8DsD20RVuuG7g6Y7F9y5EbiPHT05ug_m3eh8MBOaMDvNmk-DX4g0igF/pub?gid=1043526978&single=true&output=csv")
cleancols(df)
df.columns =lower_col(df)
df_aut = pd.melt(df,id_vars=['stt', 'tenreport', 'linkreport', 'type','id', 'id_mb', 'vw', 'param'])
df_aut=df_aut[~df_aut['value'].isna()]
df_aut.rename(columns={'value':'accessgroup'},inplace=True)

In [6]:
df_aut.head()

,stt,tenreport,linkreport,type,id,id_mb,vw,param,variable,accessgroup
0,1,MR | Sales Report - Mobile,https://datastudio.google.com/reporting/805cab...,0,805cab11-106f-414c-9d65-6e7e66a040d5,805cab11-106f-414c-9d65-6e7e66a040d5,600px,?params=xxxxxx,ag1,Phanam-ALL-CEO
1,2,NS | Sales Performance,https://datastudio.google.com/reporting/01efbd...,0,01efbde2-97eb-483f-8191-936762a38b32,c91090ba-f2ae-464e-97da-e93a316eb19e,95vw,?params=xxxxxx,ag1,Phanam-ALL-CEO
2,3,MR | Báo cáo tồn kho hằng ngày (SC và PBH),https://datastudio.google.com/reporting/4c7606...,0,4c760642-716c-4a38-8e81-2f37731da019,4c760642-716c-4a38-8e81-2f37731da019,80vw,?params=xxxxxx,ag1,Phanam-ALL-CEO
3,4,NM | Báo cáo tồn kho phục vụ sản xuất,https://datastudio.google.com/reporting/aec470...,0,aec470cf-edef-45d1-8ea1-2f801d370100,aec470cf-edef-45d1-8ea1-2f801d370100,80vw,?params=xxxxxx,ag1,Phanam-ALL-CEO
4,5,GD | Quản trị dịch vụ giao hàng,https://datastudio.google.com/reporting/42dc6b...,0,42dc6b37-46f7-4795-928c-7d922387c2a1,1138db43-64d5-4bce-b9dc-599a4ff88af2,80vw,?params=xxxxxx,ag1,Phanam-ALL-CEO


In [9]:
#lấy data phân quyền detail tách theo phân quyền CA & non CA
df_group = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vSuXl7AnpSA5j_KrgFO2zhLn8DsD20RVuuG7g6Y7F9y5EbiPHT05ug_m3eh8MBOaMDvNmk-DX4g0igF/pub?gid=2075224509&single=true&output=csv")
cleancols(df_group)
df_group.columns=lower_col(df_group)    
dk2= df_group['csv_link']!='CA'
df_link = df_group[~df_group['csv_link'].isna()]
link=df_link['csv_link'][df_link['accessgroupcongtyphongbanlevel']!='CA']
# ca_link=df_link['csv_link'][df_link['accessgroupcongtyphongbanlevel']=='CA']

# #Join phân quyền theo Access Group
df0 = pd.DataFrame()
for i in link:
    df = pd.read_csv(i)
    df0 = pd.concat([df0,df])
dropdup(df0,3)
df0['accessgroup'] = df0['congty']+'-'+df0['phongban']+'-'+df0['chucdanh']

In [11]:
df0.head()

,manv,tencvbh,email,chucdanh,congty,phongban,kenhphutrach,trangthaihoatdong,nhanvien_thaythe,note,...,AG16,AG17,AG18,AG19,AG20,AG21,AG22,AG23,AG24,accessgroup
0,MR2032,Nguyễn Thanh Sơn,thanhsonbg.vnn@gmail.com,Level1-CTV/CRS/CRSS,Trading,PBH,INS,Còn hoạt động,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Trading-PBH-Level1-CTV/CRS/CRSS
1,MR2424,Lưu Thị Hương Liễu,lieulth8889@gmail.com,Level1-CTV/CRS/CRSS,Trading,PBH,INS,Còn hoạt động,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Trading-PBH-Level1-CTV/CRS/CRSS
2,MR2561,Trần Văn Quỳnh,tranquynh526@gmail.com,Level1-CTV/CRS/CRSS,Trading,PBH,INS,Còn hoạt động,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Trading-PBH-Level1-CTV/CRS/CRSS
3,MR2588,Lê Văn Hiếu,lehieuttth@gmail.com,Level1-CTV/CRS/CRSS,Trading,PBH,INS,Còn hoạt động,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Trading-PBH-Level1-CTV/CRS/CRSS
4,MR2562,Phạm Thị Linh,phamthuylinh19052003@gmail.com,Level1-CTV/CRS/CRSS,Trading,PBH,INS,Còn hoạt động,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Trading-PBH-Level1-CTV/CRS/CRSS


In [10]:
# df0.head()

In [ ]:
full_df = pd.merge(df_aut,df0,how='inner', on='accessgroup')

In [4]:
full_df.head()

,stt,tenreport,linkreport,type,id_x,id_mb_x,vw_x,param_x,variable,accessgroup,...,AG16,AG17,AG18,AG19,AG20,AG21,AG22,AG23,AG24,inserted_at
0,1,MR | Sales Report - Mobile,https://datastudio.google.com/reporting/805cab...,0,805cab11-106f-414c-9d65-6e7e66a040d5,805cab11-106f-414c-9d65-6e7e66a040d5,600px,?params=xxxxxx,ag1,Phanam-ALL-CEO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-28 11:47:55.054671
1035,1,MR | Sales Report - Mobile,https://datastudio.google.com/reporting/805cab...,0,805cab11-106f-414c-9d65-6e7e66a040d5,805cab11-106f-414c-9d65-6e7e66a040d5,600px,?params=xxxxxx,ag4,ALL-ALL-BOD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-28 11:47:55.054671
1034,1,MR | Sales Report - Mobile,https://datastudio.google.com/reporting/805cab...,0,805cab11-106f-414c-9d65-6e7e66a040d5,805cab11-106f-414c-9d65-6e7e66a040d5,600px,?params=xxxxxx,ag4,ALL-ALL-BOD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-28 11:47:55.054671
1038,1,MR | Sales Report - Mobile,https://datastudio.google.com/reporting/805cab...,0,805cab11-106f-414c-9d65-6e7e66a040d5,805cab11-106f-414c-9d65-6e7e66a040d5,600px,?params=xxxxxx,ag4,ALL-ALL-BOD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-28 11:47:55.054671
1245,1,MR | Sales Report - Mobile,https://datastudio.google.com/reporting/805cab...,0,805cab11-106f-414c-9d65-6e7e66a040d5,805cab11-106f-414c-9d65-6e7e66a040d5,600px,?params=xxxxxx,ag3,Holding-APP-IT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-28 11:47:55.054671


In [ ]:
full_df['inserted_at']=datetime.now()
full_df=full_df.sort_values('stt')

# full_df.to_clipboard()
full_df = dropdup(full_df, 1, subset=['id', 'manv'])

In [5]:
PXKKVCNB = pd.read_csv("pr_IN_RawdataXNTByLot_BI.csv")

In [18]:
PXKKVCNB['expdate1'] = pd.to_datetime(PXKKVCNB['expdate'])

In [19]:
PXKKVCNB[PXKKVCNB.invtid == 'OH078'].to_clipboard()

In [2]:
df = pd.read_pickle("20221229_DF5_Dup.pickle")

In [3]:
df.head()

,masanpham,tensanpham,donvi,chinhanh,songaynhan,tonao,tonhangdiduong,toncn,tonhcm,tonmerap,tonvime,toncn_huy,tonhcm_huy,tonece
186,EH109,Cebest 50 Gói,HOP,HN,1,-100.0,0.0,372.0,521.0,0.0,0.0,0.0,0.0,0.0
187,EH109,Cebest 50 Gói,HOP,HN,6,-150.0,0.0,0.0,510.0,0.0,0.0,0.0,0.0,0.0


In [5]:
sql = \
"""
SELECT u.UserName,u.FirstName,sc.ScreenNumber,sc.Descr,a.SessionNumber, a.InternetAddress, a.ComputerName, a.CompanyID
FROM dbo.SYS_Access a
INNER JOIN dbo.Users u ON a.UserId=u.UserName
and u.UserTypes like '%CS%'
INNER JOIN dbo.SYS_Screen sc ON sc.ScreenNumber=a.ScreenNumber
WHERE CAST(a.AccessDate AS DATE) BETWEEN '20221201' AND '20221231'
"""
df = get_ms_df(sql)
bq_values_insert(df, "d_sync_sysaccess", 3)
